In [6]:
from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('dataset.csv')
df.head()

,uu_usl_name,CLASS_ID,SERVICE_CLASS_CONFIRMED,CNT_DIST_USL_BY_CLASS,CNT_NUM_USL
0,АМБУЛАТОРНАЯ ПОМОЩЬ,1000,NaN,12614,812414
1,АМБУЛАТОРНАЯ ПОМОЩЬ,1292,NaN,777,812414
2,КОНСУЛЬТАТИВНО-ДИАГНОСТИЧЕСКАЯ ПОМОЩЬ,1000,NaN,12614,740580
3,Сбербанк Премьер,3200,1.0,3974,489254
4,Сбербанк Премьер,8350,1.0,104,489254


In [23]:
X = df['uu_usl_name']
y = df['CLASS_ID']

In [24]:
X

0                                      АМБУЛАТОРНАЯ ПОМОЩЬ
1                                      АМБУЛАТОРНАЯ ПОМОЩЬ
2                    КОНСУЛЬТАТИВНО-ДИАГНОСТИЧЕСКАЯ ПОМОЩЬ
3                                         Сбербанк Премьер
4                                         Сбербанк Премьер
                               ...                        
81601        Прием врачом-онкологом первичный амбулаторный
81602                             Проводниковая анестезия 
81603    Проводниковая анестезия (блокада по Оберсту-Лу...
81604           Проводниковая анестезия Sol. Ultracani D.S
81605    Удаление новообразования мягких тканей под мес...
Name: uu_usl_name, Length: 81606, dtype: object

In [27]:
# from pymorphy2 import MorphAnalyzer
# morph = MorphAnalyzer()
# for i in range(X.shape[0]):
#     new_text = [[] for i in range(X.shape[0])]
#     for word in X[i].split():
#         if word.isdigit():
#             new_text[i].append(morph.parse(word)[0].normal_form)
#     X[i] = ' '.join(new_text[i])

C:\Users\user\AppData\Local\Temp\ipykernel_3440\2740834943.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = ' '.join(new_text[i])
C:\Users\user\AppData\Local\Temp\ipykernel_3440\2740834943.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = ' '.join(new_text[i])
C:\Users\user\AppData\Local\Temp\ipykernel_3440\2740834943.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = ' '.join(new_text[i])
C:\Users\user\AppDa

KeyboardInterrupt: 

In [ ]:
# x is text
# y is num
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(X)

In [18]:
# print X


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [11]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [12]:
model.score(X_test,y_test)

0.4585835069231712